# Preprocessing Dataset Pariwisata (scraped.csv)

Notebook ini melakukan preprocessing terhadap data hasil scraping dari portal berita pariwisata.

**Tahapan utama:**
1. Load `scraped.csv`
2. Hapus duplikat dan baris tidak valid
3. Gabungkan kolom teks (`title`, `description`, `location`)
4. Filter dokumen:
   - Minimal jumlah kata tertentu
   - Mengandung keyword pariwisata (`KEYWORD_FILTER` dari `config.py`)
5. Normalisasi teks (cleaning + optional stemming)
6. Simpan hasil akhir ke:
   - `data/corpus_clean.csv`
   - `data/corpus_clean.jsonl`


In [8]:
import os
import re
import json
import time

import pandas as pd

from config import KEYWORD_FILTER  # pastikan config.py ada di folder yang sama

# Path file
INPUT_PATH = "data/scraped.csv"
OUTPUT_CSV = "data/corpus_clean.csv"
OUTPUT_JSONL = "data/corpus_clean.jsonl"

# Minimal jumlah kata supaya dianggap artikel layak
MIN_WORDS = 40

print("Input  :", INPUT_PATH)
print("Output :", OUTPUT_CSV, "dan", OUTPUT_JSONL)

# Load stopwords Bahasa Indonesia
STOPWORDS = set()
stopwords_path = "stopwords_id.txt"
if os.path.exists(stopwords_path):
    with open(stopwords_path, "r", encoding="utf-8") as f:
        STOPWORDS = set(line.strip().lower() for line in f if line.strip())
    print(f"[INFO] Loaded {len(STOPWORDS)} stopwords dari {stopwords_path}")
else:
    print(f"[WARNING] File stopwords tidak ditemukan: {stopwords_path}")
    print("          Stopword removal akan di-skip")

# Optional: stemming Bahasa Indonesia dengan Sastrawi
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    print("[INFO] Sastrawi ditemukan, stemming AKTIF.")
except ImportError:
    stemmer = None
    print("[INFO] Sastrawi TIDAK ditemukan, stemming NONAKTIF.")
    print("       Jika mau pakai stemming: pip install Sastrawi")


Input  : data/scraped.csv
Output : data/corpus_clean.csv dan data/corpus_clean.jsonl
[INFO] Loaded 761 stopwords dari stopwords_id.txt
[INFO] Sastrawi ditemukan, stemming AKTIF.


## 1. Load `scraped.csv`

Di tahap ini kita:
- Membaca file CSV hasil scraping
- Melihat jumlah baris dan beberapa contoh data


In [9]:
start_time = time.time()

if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"File input tidak ditemukan: {INPUT_PATH}")

df_raw = pd.read_csv(INPUT_PATH)
print("Jumlah baris mentah:", len(df_raw))
df_raw.head()


Jumlah baris mentah: 4852


,url,domain,title,content,word_count,timestamp,image_url
0,https://travel.kompas.com/read/2025/11/11/1900...,travel.kompas.com,Unik! Kamar Hotel Bertema Kereta Api di Jepang...,KOMPAS.com– Di Jepang ternyata ada hotel denga...,485,1.763449e+09,NaN
1,https://travel.kompas.com/read/2025/11/11/1925...,travel.kompas.com,"4 Rekomendasi Wisata di Banyuwangi, Cocok untu...","KOMPAS.com -Banyuwangi, Jawa Timur kerap menja...",282,1.763449e+09,NaN
2,https://travel.kompas.com/read/2025/05/03/1333...,travel.kompas.com,"Itinerary Seharian di Bromo Jawa Timur, dari S...",Penulis\nKOMPAS.com- Wisata ke Bromo tak harus...,463,1.763449e+09,NaN
3,https://travel.kompas.com/read/2025/11/10/1851...,travel.kompas.com,"Promo Hari Pahlawan 2025, TMII Beri Diskon 30 ...",Penulis\nKOMPAS.com- Memperingati Hari Pahlawa...,362,1.763449e+09,NaN
4,https://travel.kompas.com/read/2025/11/11/1806...,travel.kompas.com,Gratis Tiket Wisata Trenggalek bagi Penumpang ...,Penulis\nKOMPAS.com– Bupati Trenggalek Mochama...,306,1.763449e+09,NaN


## 2. Bersihkan URL dan buang duplikat

Langkah:
- Buang baris tanpa `url`
- Strip spasi di `url`
- Buang baris dengan `url` kosong
- Hapus duplikat berdasarkan `url`


In [10]:
df = df_raw.copy()

# Buang baris yang tidak punya URL
df["url"] = df["url"].astype(str).str.strip()
before = len(df)
df = df[df["url"] != ""]
after_no_empty = len(df)

# Hapus duplikat berdasarkan URL
df = df.drop_duplicates(subset="url", keep="first")
after_dedup = len(df)

print(f"Total awal           : {before}")
print(f"Setelah buang kosong : {after_no_empty}")
print(f"Setelah buang duplikat URL: {after_dedup}")

df.head()


Total awal           : 4852
Setelah buang kosong : 4852
Setelah buang duplikat URL: 4852


,url,domain,title,content,word_count,timestamp,image_url
0,https://travel.kompas.com/read/2025/11/11/1900...,travel.kompas.com,Unik! Kamar Hotel Bertema Kereta Api di Jepang...,KOMPAS.com– Di Jepang ternyata ada hotel denga...,485,1.763449e+09,NaN
1,https://travel.kompas.com/read/2025/11/11/1925...,travel.kompas.com,"4 Rekomendasi Wisata di Banyuwangi, Cocok untu...","KOMPAS.com -Banyuwangi, Jawa Timur kerap menja...",282,1.763449e+09,NaN
2,https://travel.kompas.com/read/2025/05/03/1333...,travel.kompas.com,"Itinerary Seharian di Bromo Jawa Timur, dari S...",Penulis\nKOMPAS.com- Wisata ke Bromo tak harus...,463,1.763449e+09,NaN
3,https://travel.kompas.com/read/2025/11/10/1851...,travel.kompas.com,"Promo Hari Pahlawan 2025, TMII Beri Diskon 30 ...",Penulis\nKOMPAS.com- Memperingati Hari Pahlawa...,362,1.763449e+09,NaN
4,https://travel.kompas.com/read/2025/11/11/1806...,travel.kompas.com,Gratis Tiket Wisata Trenggalek bagi Penumpang ...,Penulis\nKOMPAS.com– Bupati Trenggalek Mochama...,306,1.763449e+09,NaN


## 3. Gabungkan kolom teks (`title`, `content`)

Kita bikin satu kolom baru `content_raw` berisi gabungan:
- `title`
- `content`

Kalau ada kolom yang tidak ada di CSV, akan dibuat kosong.


In [11]:
# Pastikan kolom yang diperlukan ada, kalau tidak -> isi string kosong
# PERBAIKAN: Gunakan kolom 'content' yang ada di scraped.csv, bukan 'description' dan 'location'
for col in ["title", "content", "image_url"]:
    if col not in df.columns:
        df[col] = ""

df["title"] = df["title"].fillna("").astype(str)
df["content"] = df["content"].fillna("").astype(str)
df["image_url"] = df["image_url"].fillna("").astype(str)

# Gabungkan title dan content
df["content_raw"] = (
    df["title"] + " " + df["content"]
).str.strip()

# Buang baris yang content_raw-nya bener-bener kosong
before_nonempty = len(df)
df = df[df["content_raw"].str.len() > 0]
after_nonempty = len(df)

# Hitung jumlah kata mentah
df["word_count_raw"] = df["content_raw"].str.split().str.len()

print(f"Setelah buang content kosong : {after_nonempty} (dari {before_nonempty})")
df[["url", "title", "word_count_raw"]].head()


Setelah buang content kosong : 4852 (dari 4852)



,url,title,word_count_raw
0,https://travel.kompas.com/read/2025/11/11/1900...,Unik! Kamar Hotel Bertema Kereta Api di Jepang...,497
1,https://travel.kompas.com/read/2025/11/11/1925...,"4 Rekomendasi Wisata di Banyuwangi, Cocok untu...",292
2,https://travel.kompas.com/read/2025/05/03/1333...,"Itinerary Seharian di Bromo Jawa Timur, dari S...",473
3,https://travel.kompas.com/read/2025/11/10/1851...,"Promo Hari Pahlawan 2025, TMII Beri Diskon 30 ...",378
4,https://travel.kompas.com/read/2025/11/11/1806...,Gratis Tiket Wisata Trenggalek bagi Penumpang ...,318


## 4. Filter dokumen berdasarkan minimal jumlah kata

Kita hanya keep dokumen dengan:
- `word_count_raw >= MIN_WORDS` (default 40)
Supaya yang tersisa benar-benar artikel, bukan snippet pendek.


In [12]:
before_len = len(df)
df = df[df["word_count_raw"] >= MIN_WORDS]
after_len = len(df)

print(f"Minimal {MIN_WORDS} kata:")
print(f" - Sebelum: {before_len}")
print(f" - Sesudah: {after_len}")


Minimal 40 kata:
 - Sebelum: 4852
 - Sesudah: 4850


## 5. Filter berdasarkan keyword pariwisata (`KEYWORD_FILTER`)

Hanya dokumen yang mengandung setidaknya satu kata dari `KEYWORD_FILTER`
(yang diambil dari `config.py`) yang akan dipertahankan.


In [13]:
def contains_keywords(text: str) -> bool:
    t = str(text).lower()
    return any(kw.lower() in t for kw in KEYWORD_FILTER)

df["has_keyword"] = df["content_raw"].apply(contains_keywords)

before_kw = len(df)
df = df[df["has_keyword"]]
after_kw = len(df)

print("Setelah filter keyword pariwisata:")
print(f" - Sebelum: {before_kw}")
print(f" - Sesudah: {after_kw}")
df[["url", "title", "has_keyword"]].head()


Setelah filter keyword pariwisata:
 - Sebelum: 4850
 - Sesudah: 4589


,url,title,has_keyword
0,https://travel.kompas.com/read/2025/11/11/1900...,Unik! Kamar Hotel Bertema Kereta Api di Jepang...,True
1,https://travel.kompas.com/read/2025/11/11/1925...,"4 Rekomendasi Wisata di Banyuwangi, Cocok untu...",True
2,https://travel.kompas.com/read/2025/05/03/1333...,"Itinerary Seharian di Bromo Jawa Timur, dari S...",True
3,https://travel.kompas.com/read/2025/11/10/1851...,"Promo Hari Pahlawan 2025, TMII Beri Diskon 30 ...",True
4,https://travel.kompas.com/read/2025/11/11/1806...,Gratis Tiket Wisata Trenggalek bagi Penumpang ...,True


## 6. Normalisasi teks (`content_clean`)

Tahap cleaning:
- Ubah ke lowercase
- Hapus URL
- Hapus karakter non-alfanumerik (selain spasi)
- Rapikan spasi
- **Hapus stopwords** (kata-kata umum yang tidak informatif)
- (Opsional) Stemming Bahasa Indonesia dengan Sastrawi

Hasilnya disimpan di kolom `content_clean`.


In [14]:
url_pattern = re.compile(r"https?://\S+")
non_alnum_pattern = re.compile(r"[^0-9a-zA-Z\u00C0-\u024F\u1E00-\u1EFF ]+")

def clean_text(text: str) -> str:
    txt = str(text).lower()

    # Hapus URL
    txt = url_pattern.sub(" ", txt)

    # Hapus karakter non-alfanumerik (kecuali spasi)
    txt = non_alnum_pattern.sub(" ", txt)

    # Rapikan spasi
    txt = re.sub(r"\s+", " ", txt).strip()

    # Tokenisasi untuk stopword removal
    tokens = txt.split()
    
    # Hapus stopwords jika tersedia
    if STOPWORDS:
        tokens = [t for t in tokens if t not in STOPWORDS]
    
    # Gabungkan kembali sebelum stemming
    txt = " ".join(tokens)

    # Stemming kalau stemmer tersedia
    if stemmer is not None and txt:
        txt = stemmer.stem(txt)

    return txt

df["content_clean"] = df["content_raw"].apply(clean_text)
df["word_count_clean"] = df["content_clean"].str.split().str.len()

# Buang baris yang setelah cleaning jadi kosong
before_clean_nonempty = len(df)
df = df[df["word_count_clean"] > 0]
after_clean_nonempty = len(df)

print(f"Setelah cleaning & buang yang kosong:")
print(f" - Sebelum: {before_clean_nonempty}")
print(f" - Sesudah: {after_clean_nonempty}")

df[["url", "word_count_raw", "word_count_clean"]].head()


KeyboardInterrupt: 

## 7. Simpan hasil preprocessing

Kita simpan dua versi:
- `corpus_clean.csv` → gampang dicek pakai Excel/Spreadsheet
- `corpus_clean.jsonl` → enak dipakai untuk indexing / IR engine


In [ ]:
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)

# Kolom yang mau disimpan
cols = [
    "url",
    "title",
    "image_url",
    "word_count_raw",
    "word_count_clean",
    "content_raw",
    "content_clean",
]

# CSV
df.to_csv(OUTPUT_CSV, index=False, columns=cols)
print(f"[SAVE] CSV   -> {OUTPUT_CSV}")

# JSONL
with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
    for _, row in df[cols].iterrows():
        rec = {col: row[col] for col in cols}
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
print(f"[SAVE] JSONL -> {OUTPUT_JSONL}")

elapsed = time.time() - start_time
minutes = int(elapsed // 60)
seconds = int(elapsed % 60)
print(f"\n=== PREPROCESSING SELESAI ===")
print(f"Total dokumen bersih: {len(df)}")
print(f"Waktu proses         : {minutes} menit {seconds} detik (~{elapsed:.2f} s)")


[SAVE] CSV   -> data/corpus_clean.csv
[SAVE] JSONL -> data/corpus_clean.jsonl

=== PREPROCESSING SELESAI ===
Total dokumen bersih: 4700
Waktu proses         : 58 menit 33 detik (~3513.69 s)
[SAVE] JSONL -> data/corpus_clean.jsonl

=== PREPROCESSING SELESAI ===
Total dokumen bersih: 4700
Waktu proses         : 58 menit 33 detik (~3513.69 s)
